***GENERATED CODE FOR schedulegbafescdb PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AppointmentID', 'transformation_label': 'String Indexer'}], 'feature': 'AppointmentID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '00DD0705-E57A-487A-8F02-6BB1E2EBADF5', 'max': 'FFC30CDC-18B5-48F7-ABF6-22E674198733', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AppointmentID'}, {'feature_label': 'AppointmentID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AppointmentID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PatientNumber', 'transformation_label': 'String Indexer'}], 'feature': 'PatientNumber', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '426271.49', 'stddev': '189765.94', 'min': '        3571', 'max': '      604978', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PatientNumber'}, {'feature_label': 'PatientNumber', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PatientNumber')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'PatientLoadDate', 'transformation_label': 'Extract Date'}], 'feature': 'PatientLoadDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'PatientLoadDate'}, {'feature_label': 'PatientLoadDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('PatientLoadDate')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'AppointmentDate', 'transformation_label': 'Extract Date'}], 'feature': 'AppointmentDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'AppointmentDate'}, {'feature_label': 'AppointmentDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('AppointmentDate')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'AppointmentCreatedDate', 'transformation_label': 'Extract Date'}], 'feature': 'AppointmentCreatedDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'AppointmentCreatedDate'}, {'feature_label': 'AppointmentCreatedDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('AppointmentCreatedDate')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'EncounterDate', 'transformation_label': 'Extract Date'}], 'feature': 'EncounterDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'EncounterDate'}, {'feature_label': 'EncounterDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('EncounterDate')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'ModifiedDate', 'transformation_label': 'Extract Date'}], 'feature': 'ModifiedDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'ModifiedDate'}, {'feature_label': 'ModifiedDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('ModifiedDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AppointmentStatus', 'transformation_label': 'String Indexer'}], 'feature': 'AppointmentStatus', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Kept', 'max': 'Kept', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AppointmentStatus'}, {'feature_label': 'AppointmentStatus', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AppointmentStatus')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AppointmnetType', 'transformation_label': 'String Indexer'}], 'feature': 'AppointmnetType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '*ESTABLISHED PATIENT', 'max': 'THERAPY', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AppointmnetType'}, {'feature_label': 'AppointmnetType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AppointmnetType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'RescheduleFlag', 'transformation_label': 'String Indexer'}], 'feature': 'RescheduleFlag', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'N', 'max': 'Y', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'RescheduleFlag'}, {'feature_label': 'RescheduleFlag', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('RescheduleFlag')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PracticeName', 'transformation_label': 'String Indexer'}], 'feature': 'PracticeName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'MS PG1', 'max': 'MS PG1', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PracticeName'}, {'feature_label': 'PracticeName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PracticeName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PracticeSpeciality', 'transformation_label': 'String Indexer'}], 'feature': 'PracticeSpeciality', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Multi-Specialty', 'max': 'Multi-Specialty', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PracticeSpeciality'}, {'feature_label': 'PracticeSpeciality', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PracticeSpeciality')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PracticeRegion', 'transformation_label': 'String Indexer'}], 'feature': 'PracticeRegion', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'SouthEast', 'max': 'SouthEast', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PracticeRegion'}, {'feature_label': 'PracticeRegion', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PracticeRegion')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'SocCLEXX', 'max': 'SocUniXX', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityCategory'}, {'feature_label': 'FacilityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityCity', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityCity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'HOUSTON', 'max': 'Sugar Land', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityCity'}, {'feature_label': 'FacilityCity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityCity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityState', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityState', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'TX', 'max': 'TX', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityState'}, {'feature_label': 'FacilityState', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityState')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityZip', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityZip', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '771662357.01', 'stddev': '1749501.14', 'min': '770025351', 'max': '774792919', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityZip'}, {'feature_label': 'FacilityZip', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityZip')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderName', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'NoelAhmyyy, HassaXXX', 'max': 'NoelYouyyy, MeagaXXX', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderName'}, {'feature_label': 'ProviderName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Speciality', 'transformation_label': 'String Indexer'}], 'feature': 'Speciality', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Allergy/Immun', 'max': 'Otolaryngology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Speciality'}, {'feature_label': 'Speciality', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Speciality')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderTaxonomyName', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderTaxonomyName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Allopathic & Osteopathic Physicians : Allergy & Immunology', 'max': 'Unspecified', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderTaxonomyName'}, {'feature_label': 'ProviderTaxonomyName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderTaxonomyName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderGroup1', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderGroup1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Allergy', 'max': 'Unspecified', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderGroup1'}, {'feature_label': 'ProviderGroup1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderGroup1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderGroup2', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderGroup2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Allergy', 'max': 'Unspecified', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderGroup2'}, {'feature_label': 'ProviderGroup2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderGroup2')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderCreatedDate', 'transformation_label': 'Extract Date'}], 'feature': 'ProviderCreatedDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'ProviderCreatedDate'}, {'feature_label': 'ProviderCreatedDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('ProviderCreatedDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReferringProviderName', 'transformation_label': 'String Indexer'}], 'feature': 'ReferringProviderName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Abxxxdia', 'max': 'Zoxxxene', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReferringProviderName'}, {'feature_label': 'ReferringProviderName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReferringProviderName')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split


def naiveBayes(df, labels, features, TrainingPercent=100):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    clf = GaussianNB()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    clf.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % clf.score(X_test, y_test))

    data = {'model': clf,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run schedulegbafescdbHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'IntelligentScheduling', 'optionalDB': 'IntelligentScheduling', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT * FROM\r\n(\r\n\r\n\t\t\tSELECT DISTINCT\r\n\t\t\tA.AppointmentID,\r\n\t\t\tP.PatientNumber, P.Gender, P.LoadDate as PatientLoadDate,\r\n\t\t\tFLOOR(DATEDIFF(DAY, P.DateOfBirth, GETDATE()) / 365) AS Age,\r\n\t\t\tA.AppointmentDate,A.CreatedDate as AppointmentCreatedDate,EN.EncounterDate,\r\n\t\t\tA.ModifiedDate,\r\n\t\t\tDATEDIFF(day, A.CreatedDate,A.AppointmentDate) as days_AppDate_AppCreatDate, \r\n\t\t\tDATEDIFF(day, A.CreatedDate,A.ModifiedDate) as days_AppCreatDate_AppModiDate, \r\n\t\t\tDATEDIFF(day, A.AppointmentDate,A.ModifiedDate) as days_AppDate_AppModiDate, \r\n\t\t\tA.AppointmentDuration,\r\n\t\t\tS.AppointmentStatus,\r\n\t\t\tAT.AppointmentTypeCategory as AppointmnetType,\r\n\t\t\tA.RescheduleFlag, A.EligibilityFlag, A.ConfirmationFlag, \r\n\t\t\tPRC.PracticeName, PRC.Speciality as PracticeSpeciality, PRC.Region as PracticeRegion,\r\n\t\t\tF.FacilityCategory, F.City as FacilityCity,F.State as FacilityState,F.Zip as FacilityZip,\r\n\t\t\tPR.ProviderCategory as ProviderName, SP.SpecialityCategory as Speciality, PTx.ProviderTaxonomyCategory as ProviderTaxonomyName,\r\n\t\t\tPR.ProviderGroup1, PR.ProviderGroup2, PR.LoadDate as ProviderCreatedDate,\r\n\t\t\tRP.ReferringProviderCategory as ReferringProviderName,\r\n\t\t\tLabel=0\r\n\r\n\t\t\tFROM APPOINTMENT A\r\n\t\t\tINNER JOIN  APPOINTMENTSTATUS  S on S.AppointmentStatusID=A.AppointmentStatusID\r\n\t\t\tINNER JOIN APPOINTMENTTYPE AT ON AT.AppointmentTypeID=A.AppointmentTypeID\r\n\t\t\tINNER JOIN ENCOUNTER EN ON EN.EncounterID=A.EncounterID\r\n\t\t\tINNER JOIN ENCOUNTERSTATUS ES ON ES.EncounterStatusID=EN.EncounterStatusID\r\n\t\t\tINNER JOIN PATIENT P ON P.PatientID=A.PatientID\r\n\t\t\tINNER JOIN PROVIDER PR ON PR.ProviderID=A.ProviderID\r\n\t\t\tINNER JOIN ProviderTaxonomy PTx ON PR.TaxonomyID=PTx.ProviderTaxonomyID\r\n\t\t\tINNER JOIN ReferringProvider RP ON RP.ReferringProviderID=A.ReferringProviderID\r\n\t\t\tINNER JOIN SPECIALITY SP ON SP.SpecialityID=PR.SpecialityID\r\n\t\t\tINNER JOIN FACILITY F ON F.Facilityid=A.FacilityID\r\n\t\t\tINNER JOIN PRACTICE PRC ON PRC.PracticeID=A.PracticeID\r\n\t\t\tWHERE\r\n\t\t\tA.AppointmentDate between  '1/01/2020' and '1/31/2021'\r\n\t\t\tAND A.RescheduleFlag IN ('Y')\r\nUNION\r\n\r\nSELECT DISTINCT\r\n\r\n\tA.AppointmentID,\r\n\tP.PatientNumber, P.Gender, P.LoadDate as PatientLoadDate,\r\n\tFLOOR(DATEDIFF(DAY, P.DateOfBirth, GETDATE()) / 365) AS Age,\r\n\tA.AppointmentDate,A.CreatedDate as AppointmentCreatedDate,EN.EncounterDate,\r\n\tA.ModifiedDate,\r\n\tDATEDIFF(day, A.CreatedDate,A.AppointmentDate) as days_AppDate_AppCreatDate, \r\n\tDATEDIFF(day, A.CreatedDate,A.ModifiedDate) as days_AppCreatDate_AppModiDate, \r\n\tDATEDIFF(day, A.AppointmentDate,A.ModifiedDate) as days_AppDate_AppModiDate, \r\n\tA.AppointmentDuration,\r\n\tS.AppointmentStatus,\r\n\tAT.AppointmentTypeCategory as AppointmnetType,\r\n\tA.RescheduleFlag, A.EligibilityFlag, A.ConfirmationFlag, \r\n\tPRC.PracticeName, PRC.Speciality as PracticeSpeciality, PRC.Region as PracticeRegion,\r\n\tF.FacilityCategory, F.City as FacilityCity,F.State as FacilityState,F.Zip as FacilityZip,\r\n\tPR.ProviderCategory as ProviderName, SP.SpecialityCategory as Speciality, PTx.ProviderTaxonomyCategory as ProviderTaxonomyName,\r\n\tPR.ProviderGroup1, PR.ProviderGroup2, PR.LoadDate as ProviderCreatedDate,\r\n\tRP.ReferringProviderCategory as ReferringProviderName,\r\n\tLabel=1\r\n\r\n\tFROM APPOINTMENT A\r\n\tINNER JOIN  APPOINTMENTSTATUS  S on S.AppointmentStatusID=A.AppointmentStatusID\r\n\tINNER JOIN APPOINTMENTTYPE AT ON AT.AppointmentTypeID=A.AppointmentTypeID\r\n\tINNER JOIN ENCOUNTER EN ON EN.EncounterID=A.EncounterID\r\n\tINNER JOIN ENCOUNTERSTATUS ES ON ES.EncounterStatusID=EN.EncounterStatusID\r\n\tINNER JOIN PATIENT P ON P.PatientID=A.PatientID\r\n\tINNER JOIN PROVIDER PR ON PR.ProviderID=A.ProviderID\r\n\tINNER JOIN ProviderTaxonomy PTx ON PR.TaxonomyID=PTx.ProviderTaxonomyID\r\n\tINNER JOIN ReferringProvider RP ON RP.ReferringProviderID=A.ReferringProviderID\r\n\tINNER JOIN SPECIALITY SP ON SP.SpecialityID=PR.SpecialityID\r\n\tINNER JOIN FACILITY F ON F.Facilityid=A.FacilityID\r\n\tINNER JOIN PRACTICE PRC ON PRC.PracticeID=A.PracticeID\r\n\tWHERE\r\n\tA.AppointmentDate between  '1/01/2020' and '1/31/2021'\r\n\tAND A.RescheduleFlag NOT IN ('Y')\r\n) as ACD", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run schedulegbafescdbHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "AppointmentID", "transformation_label": "String Indexer"}], "feature": "AppointmentID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "00DD0705-E57A-487A-8F02-6BB1E2EBADF5", "max": "FFC30CDC-18B5-48F7-ABF6-22E674198733", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AppointmentID"}, {"transformationsData": [{"feature_label": "PatientNumber", "transformation_label": "String Indexer"}], "feature": "PatientNumber", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "426271.49", "stddev": "189765.94", "min": "        3571", "max": "      604978", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientNumber"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "PatientLoadDate", "transformation_label": "Extract Date"}], "feature": "PatientLoadDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "PatientLoadDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "56.12", "stddev": "21.12", "min": "2", "max": "95", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "AppointmentDate", "transformation_label": "Extract Date"}], "feature": "AppointmentDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "AppointmentDate"}, {"transformationsData": [{"feature_label": "AppointmentCreatedDate", "transformation_label": "Extract Date"}], "feature": "AppointmentCreatedDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "AppointmentCreatedDate"}, {"transformationsData": [{"feature_label": "EncounterDate", "transformation_label": "Extract Date"}], "feature": "EncounterDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "EncounterDate"}, {"transformationsData": [{"feature_label": "ModifiedDate", "transformation_label": "Extract Date"}], "feature": "ModifiedDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "ModifiedDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "days_AppDate_AppCreatDate", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "9.48", "stddev": "19.25", "min": "0", "max": "185", "missing": "0"}, "updatedLabel": "days_AppDate_AppCreatDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "days_AppCreatDate_AppModiDate", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "16.1", "stddev": "36.45", "min": "0", "max": "441", "missing": "0"}, "updatedLabel": "days_AppCreatDate_AppModi..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "days_AppDate_AppModiDate", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "6.62", "stddev": "31.35", "min": "0", "max": "409", "missing": "0"}, "updatedLabel": "days_AppDate_AppModiDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AppointmentDuration", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "13.03", "stddev": "20.54", "min": "5", "max": "240", "missing": "0"}, "updatedLabel": "AppointmentDuration"}, {"transformationsData": [{"feature_label": "AppointmentStatus", "transformation_label": "String Indexer"}], "feature": "AppointmentStatus", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Kept", "max": "Kept", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AppointmentStatus"}, {"transformationsData": [{"feature_label": "AppointmnetType", "transformation_label": "String Indexer"}], "feature": "AppointmnetType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "*ESTABLISHED PATIENT", "max": "THERAPY", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AppointmnetType"}, {"transformationsData": [{"feature_label": "RescheduleFlag", "transformation_label": "String Indexer"}], "feature": "RescheduleFlag", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "N", "max": "Y", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "RescheduleFlag"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EligibilityFlag", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "EligibilityFlag"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ConfirmationFlag", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.63", "stddev": "0.48", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "ConfirmationFlag"}, {"transformationsData": [{"feature_label": "PracticeName", "transformation_label": "String Indexer"}], "feature": "PracticeName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "MS PG1", "max": "MS PG1", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PracticeName"}, {"transformationsData": [{"feature_label": "PracticeSpeciality", "transformation_label": "String Indexer"}], "feature": "PracticeSpeciality", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Multi-Specialty", "max": "Multi-Specialty", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PracticeSpeciality"}, {"transformationsData": [{"feature_label": "PracticeRegion", "transformation_label": "String Indexer"}], "feature": "PracticeRegion", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "SouthEast", "max": "SouthEast", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PracticeRegion"}, {"transformationsData": [{"feature_label": "FacilityCategory", "transformation_label": "String Indexer"}], "feature": "FacilityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "SocCLEXX", "max": "SocUniXX", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityCategory"}, {"transformationsData": [{"feature_label": "FacilityCity", "transformation_label": "String Indexer"}], "feature": "FacilityCity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "HOUSTON", "max": "Sugar Land", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityCity"}, {"transformationsData": [{"feature_label": "FacilityState", "transformation_label": "String Indexer"}], "feature": "FacilityState", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "TX", "max": "TX", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityState"}, {"transformationsData": [{"feature_label": "FacilityZip", "transformation_label": "String Indexer"}], "feature": "FacilityZip", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "771662357.01", "stddev": "1749501.14", "min": "770025351", "max": "774792919", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityZip"}, {"transformationsData": [{"feature_label": "ProviderName", "transformation_label": "String Indexer"}], "feature": "ProviderName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "NoelAhmyyy, HassaXXX", "max": "NoelYouyyy, MeagaXXX", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderName"}, {"transformationsData": [{"feature_label": "Speciality", "transformation_label": "String Indexer"}], "feature": "Speciality", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Allergy/Immun", "max": "Otolaryngology", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Speciality"}, {"transformationsData": [{"feature_label": "ProviderTaxonomyName", "transformation_label": "String Indexer"}], "feature": "ProviderTaxonomyName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Allopathic & Osteopathic Physicians : Allergy & Immunology", "max": "Unspecified", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderTaxonomyName"}, {"transformationsData": [{"feature_label": "ProviderGroup1", "transformation_label": "String Indexer"}], "feature": "ProviderGroup1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Allergy", "max": "Unspecified", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderGroup1"}, {"transformationsData": [{"feature_label": "ProviderGroup2", "transformation_label": "String Indexer"}], "feature": "ProviderGroup2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Allergy", "max": "Unspecified", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderGroup2"}, {"transformationsData": [{"feature_label": "ProviderCreatedDate", "transformation_label": "Extract Date"}], "feature": "ProviderCreatedDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "ProviderCreatedDate"}, {"transformationsData": [{"feature_label": "ReferringProviderName", "transformation_label": "String Indexer"}], "feature": "ReferringProviderName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Abxxxdia", "max": "Zoxxxene", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReferringProviderName"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Label", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.0", "stddev": "0.04", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Label"}, {"feature": "AppointmentID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "249.5", "stddev": "144.48", "min": "0.0", "max": "499.0", "missing": "0"}, "updatedLabel": "AppointmentID_stringindex..."}, {"feature": "PatientNumber_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "181.3", "stddev": "132.4", "min": "0.0", "max": "422.0", "missing": "0"}, "updatedLabel": "PatientNumber_stringindex..."}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.43", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "PatientLoadDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "20.75", "stddev": "4.68", "min": "1", "max": "30", "missing": "0"}, "updatedLabel": "PatientLoadDate_dayofmont..."}, {"feature": "PatientLoadDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "5.68", "stddev": "1.76", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "PatientLoadDate_month"}, {"feature": "PatientLoadDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "2020.01", "stddev": "0.09", "min": "2020", "max": "2021", "missing": "0"}, "updatedLabel": "PatientLoadDate_year"}, {"feature": "AppointmentDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "15.28", "stddev": "8.24", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "AppointmentDate_dayofmont..."}, {"feature": "AppointmentDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "6.4", "stddev": "3.98", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "AppointmentDate_month"}, {"feature": "AppointmentDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "2020.1", "stddev": "0.31", "min": "2020", "max": "2021", "missing": "0"}, "updatedLabel": "AppointmentDate_year"}, {"feature": "AppointmentCreatedDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "15.75", "stddev": "8.81", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "AppointmentCreatedDate_da..."}, {"feature": "AppointmentCreatedDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "6.7", "stddev": "3.87", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "AppointmentCreatedDate_mo..."}, {"feature": "AppointmentCreatedDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "2020.05", "stddev": "0.33", "min": "2019", "max": "2021", "missing": "0"}, "updatedLabel": "AppointmentCreatedDate_ye..."}, {"feature": "EncounterDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "15.28", "stddev": "8.24", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "EncounterDate_dayofmonth"}, {"feature": "EncounterDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "6.4", "stddev": "3.98", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "EncounterDate_month"}, {"feature": "EncounterDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "2020.1", "stddev": "0.31", "min": "2020", "max": "2021", "missing": "0"}, "updatedLabel": "EncounterDate_year"}, {"feature": "ModifiedDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "15.15", "stddev": "8.36", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "ModifiedDate_dayofmonth"}, {"feature": "ModifiedDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "6.36", "stddev": "3.94", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "ModifiedDate_month"}, {"feature": "ModifiedDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "2020.13", "stddev": "0.33", "min": "2020", "max": "2021", "missing": "0"}, "updatedLabel": "ModifiedDate_year"}, {"feature": "AppointmentStatus_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "AppointmentStatus_stringi..."}, {"feature": "AppointmnetType_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "2.39", "stddev": "3.93", "min": "0.0", "max": "23.0", "missing": "0"}, "updatedLabel": "AppointmnetType_stringind..."}, {"feature": "RescheduleFlag_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.04", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "RescheduleFlag_stringinde..."}, {"feature": "PracticeName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PracticeName_stringindexe..."}, {"feature": "PracticeSpeciality_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PracticeSpeciality_string..."}, {"feature": "PracticeRegion_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PracticeRegion_stringinde..."}, {"feature": "FacilityCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "3.12", "stddev": "2.59", "min": "0.0", "max": "13.0", "missing": "0"}, "updatedLabel": "FacilityCategory_stringin..."}, {"feature": "FacilityCity_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "1.08", "stddev": "1.06", "min": "0.0", "max": "4.0", "missing": "0"}, "updatedLabel": "FacilityCity_stringindexe..."}, {"feature": "FacilityState_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "FacilityState_stringindex..."}, {"feature": "FacilityZip_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "3.3", "stddev": "2.88", "min": "0.0", "max": "14.0", "missing": "0"}, "updatedLabel": "FacilityZip_stringindexer..."}, {"feature": "ProviderName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "9.32", "stddev": "8.96", "min": "0.0", "max": "37.0", "missing": "0"}, "updatedLabel": "ProviderName_stringindexe..."}, {"feature": "Speciality_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.28", "stddev": "0.56", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "Speciality_stringindexer_..."}, {"feature": "ProviderTaxonomyName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "1.16", "stddev": "1.8", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "ProviderTaxonomyName_stri..."}, {"feature": "ProviderGroup1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.28", "stddev": "0.56", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "ProviderGroup1_stringinde..."}, {"feature": "ProviderGroup2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.25", "stddev": "0.59", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "ProviderGroup2_stringinde..."}, {"feature": "ProviderCreatedDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "21.95", "stddev": "0.5", "min": "17", "max": "22", "missing": "0"}, "updatedLabel": "ProviderCreatedDate_dayof..."}, {"feature": "ProviderCreatedDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "5.01", "stddev": "0.1", "min": "5", "max": "6", "missing": "0"}, "updatedLabel": "ProviderCreatedDate_month"}, {"feature": "ProviderCreatedDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}, "updatedLabel": "ProviderCreatedDate_year"}, {"feature": "ReferringProviderName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "83.53", "stddev": "82.75", "min": "0.0", "max": "271.0", "missing": "0"}, "updatedLabel": "ReferringProviderName_str..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run schedulegbafescdbHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=naiveBayes(autofe, ["AppointmentID_stringindexer", "PatientNumber_stringindexer", "Gender_stringindexer", "Age", "days_AppDate_AppCreatDate", "days_AppCreatDate_AppModiDate", "days_AppDate_AppModiDate", "AppointmentDuration", "AppointmentStatus_stringindexer", "AppointmnetType_stringindexer", "RescheduleFlag_stringindexer", "EligibilityFlag", "ConfirmationFlag", "PracticeName_stringindexer", "PracticeSpeciality_stringindexer", "PracticeRegion_stringindexer", "FacilityCategory_stringindexer", "FacilityCity_stringindexer", "FacilityState_stringindexer", "FacilityZip_stringindexer", "ProviderName_stringindexer", "Speciality_stringindexer", "ProviderTaxonomyName_stringindexer", "ProviderGroup1_stringindexer", "ProviderGroup2_stringindexer", "ReferringProviderName_stringindexer", "PatientLoadDate_dayofmonth", "PatientLoadDate_month", "PatientLoadDate_year", "AppointmentDate_dayofmonth", "AppointmentDate_month", "AppointmentDate_year", "AppointmentCreatedDate_dayofmonth", "AppointmentCreatedDate_month", "AppointmentCreatedDate_year", "EncounterDate_dayofmonth", "EncounterDate_month", "EncounterDate_year", "ModifiedDate_dayofmonth", "ModifiedDate_month", "ModifiedDate_year", "ProviderCreatedDate_dayofmonth", "ProviderCreatedDate_month", "ProviderCreatedDate_year"], "Label")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
